In [ ]:
from torch import nn

# this prints out the named parameters of a model
def print_named_params(model: nn.Module) -> None:
    for name, param in model.named_parameters():
        print(f"{name}: {param.shape}")

def output_targets(model: nn.Module, file: str) -> None:
    with open(file, 'w') as f:
        for name, param in model.named_parameters():
            f.write(f"{name}\n")

In [ ]:
import torch
from copy import deepcopy
#import timm
torch.set_printoptions(precision = 6, sci_mode = False)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

In [ ]:
import sys
sys.path.append('../../pytei')

In [ ]:
def get_num_parameters(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
from pytei import Injector
def inject_error(model: nn.Module, error_map_file: str, prob) -> nn.Module:
    model_error = deepcopy(model).to(device)
    injector = Injector(error_map_file, p = prob, device = device, verbose = True)
    injector.inject(model_error)
    return model_error

## GPT 2

In [ ]:
pip install transformers

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
from collections import OrderedDict

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
gpt2 = GPT2Model.from_pretrained('gpt2-medium')

def get_modified_state_dictGPT2(model: nn.Module):
    new_state_dict = deepcopy(model.state_dict())
    for key in list(new_state_dict.keys()):
        new_state_dict[f"transformer.{key}"] = new_state_dict.pop(key)
    return new_state_dict

In [ ]:
print_named_params(gpt2)
output_targets(gpt2, "gpt2_targets")

In [ ]:
gpt2_error = inject_error(gpt2, "gpt2_targets")
gpt2_error.eval()
text = "blahblahblah"
test_input = tokenizer(text, return_tensors='pt').to(device)


with torch.no_grad():
    error_out = gpt2_error(**test_input) # gpt

## MAMBA

## Evaluate

In [ ]:
pip install deepeval

In [ ]:
from transformers import AutoModelForCausalLM,  AutoTokenizer
from deepeval.models.base_model import DeepEvalBaseLLM
from typing import List
from pydantic import BaseModel
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer.integrations.transformers import (
    build_transformers_prefix_allowed_tokens_fn,
)
import transformers
from transformers import pipeline
import json

class GPT2(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        # Same as the previous example above
        model = self.load_model()
        pipeline = transformers.pipeline(
            "text-generation",
            model=model,
            tokenizer=self.tokenizer,
            use_cache=True,
            device_map="auto",
            max_new_tokens=100,
            do_sample=True,
            top_k=5,
            num_return_sequences=1,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,
        )

        # Create parser required for JSON confinement using lmformatenforcer
        parser = JsonSchemaParser(schema.schema())
        prefix_function = build_transformers_prefix_allowed_tokens_fn(
            pipeline.tokenizer, parser
        )

        # Output and load valid JSON
        output_dict = pipeline(prompt, prefix_allowed_tokens_fn=prefix_function)
        output = output_dict[0]["generated_text"][len(prompt) :]
        json_result = json.loads(output)

        # Return valid JSON object according to the schema DeepEval supplied
        return schema(**json_result)

    async def a_generate(self, prompt: str, schema) -> BaseModel:
        return self.generate(prompt, schema)

    # This is optional.
    def batch_generate(self, promtps: List[str]) -> List[str]:
        model = self.load_model()
        device = "cuda" # the device to load the model onto

        model_inputs = self.tokenizer(promtps, return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)

    def get_model_name(self):
        return "GPT2"

#model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2") # Can be replaced with any huggingface model
#tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")



In [ ]:
from transformers import AutoModelForCausalLM, AutoConfig
from deepeval.benchmarks import MMLU
from deepeval.benchmarks.tasks import MMLUTask

'''
def convert_torch_to_huggingface(model: torch.nn.Module):
    config = AutoConfig.from_pretrained("gpt2") # Change to whichever model architecture being used
    hf_model = AutoModelForCausalLM.from_config(config)    
    hf_model.load_state_dict(model.state_dict(), strict=False)
    return hf_model
'''

def convert_torch_to_huggingface_stateGPT2(state_dict):
    config = AutoConfig.from_pretrained("gpt2-medium") # Change to whichever model architecture being used
    hf_model = AutoModelForCausalLM.from_config(config)    
    hf_model.load_state_dict(state_dict, strict=False)
    return hf_model

def evaluate_model_MMLU(model):

    benchmark = MMLU(
        tasks=[MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE, MMLUTask.ASTRONOMY],
        n_shots=3
    )

    benchmark.evaluate(model=model)
    return benchmark.task_scores

def convert_to_hf_GPT2(model: nn.Module):
    return convert_torch_to_huggingface_stateGPT2(get_modified_state_dictGPT2(model))
    
def evaluate_model_MMLU_GPT2(model: nn.Module):
    hf_model = convert_to_hf_GPT2(model)
    tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
    test_model = GPT2(model=hf_model, tokenizer=tokenizer)
    return evaluate_model_MMLU(test_model)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("gpt2")
output_targets(model, "gpt2_targets")

In [ ]:
print_named_targets(model)

In [ ]:
inject_error(model, "gpt2_targets", prob)

In [ ]:
from transformers import BitsAndBytesConfig

model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token, trust_remote_code=True)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        token=token,
    )
mistral = GPT2(model, tokenizer)

In [ ]:
probability = [2e-9, 4e-9, 6e-9, 8e-9]
for prob in probability:
    while True:
        test_model = inject_error(gpt2, "gpt2_targets", prob)
        try:
            result = evaluate_model_MMLU_GPT2(test_model)
            for i in result.index:
                task = result.loc[i, "Task"]
                score = result.loc[i, "Score"]
                with open(f"results/gpt2_{str(prob)}_{task}", "w") as f:
                    f.write(f"{str(score)}\n") 
            print("Succeeded no nan")
            break
        except:
            print("Got an error")
            pass

In [ ]:
test_model = gpt2
result = evaluate_model_MMLU_GPT2(test_model)
for i in result.index:
    task = result.loc[i, "Task"]
    score = result.loc[i, "Score"]
    with open(f"results/gpt2_{str(0)}_{task}", "w") as f:
        f.write(f"{str(score)}\n")        

In [ ]:
result = evaluate_model_MMLU_GPT2(gpt2_error)

In [ ]:
result

In [ ]:
result.loc[result["Task"] == "high_school_computer_science"]

In [ ]:
len(result)

In [ ]:
for i in result.index:
    task = result.loc[i, "Task"]
    score = result.loc[i, "Score"]
    with open(f"results/gpt2_{str(prob)}_{task}", "w") as f:
        f.write(f"{str(score)}\n") 
    print("Succeeded no nan")

In [ ]:
import matplotlib as plt